In [ ]:
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
# Khai báo các tham số của mô hình
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Các tham số của mạng
n_hidden_1 = 300 # 1st layer number of neurals
n_hidden_2 = 100 # 2nd layer number of neurals
input_shape = 784 # MNIST data input (shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, input_shape])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
# Tạo model nối các layer
def neural_net(x):
    with tf.name_scope("model"):    
        # Hidden fully connected layer with 256 neurons
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        # Hidden fully connected layer with 256 neurons
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        # Output fully connected layer with a neuron for each class
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        return out_layer

In [ ]:
# Khởi tạo weight và bias
weights = {
    'h1': tf.Variable(tf.random_normal([input_shape, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
# Xây dựng model hoàn chỉnh
logits = neural_net(X)

# Khởi tạo loss và optimizer
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
    loss_op = tf.reduce_mean(entropy, name="loss")

# Train model
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

# Đánh giá model
with tf.name_scope("evaluate"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) 
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"

logdir = "{}/run-{}/".format(root_logdir, now)

In [ ]:
# Khởi tạo các biến của graph
init = tf.global_variables_initializer()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
# Train model
with tf.Session() as sess:
    sess.run(init)
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if (step % display_step == 0 or step == 1):
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
    print("\nTesting Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
    print("Done!")

In [ ]:
file_writer.close()